In [2]:
import os
import sys
import json

# in order to recognize the utils, I had to add project root to sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

if project_root not in sys.path:
    sys.path.append(project_root)

from utils.cfd_utils import run_cfd_discovery
from utils.cfd_utils import *
from preprocessing.preprocessing_pipeline import *


In [3]:
config_path_CFD = os.path.join(project_root, 'configs', 'CFD_config.json')
cleaned_data_for_cfd = pd.read_csv('../outputs/cleaned_data_for_cfd.csv')
input_csv = os.path.join(project_root, 'outputs', 'cleaned_data_for_cfd.csv')
output_txt = os.path.join(project_root, 'outputs', 'raw_cfd.txt')


with open(config_path_CFD, 'r') as f:
     config_CFD = json.load(f)


condlhs  = config_CFD["CFDParsing"]["conditionslhs"]
condrhs  = config_CFD["CFDParsing"]["conditionsrhs"]


In [ ]:
#running the CFDDiscovery algorithm to extract raw dependencies based on parameter in CFD_config.
run_cfd_discovery(config_path_CFD, input_csv, output_txt)
raw_cfds = load_raw_cfds(output_txt) 



def filter_cfds(raw_cfds, config_CFD):
    lines = filter_by_target_value(raw_cfds, config_CFD, print_summary=False)
    lines = replace_incomparable_symbols(lines)  
    lines = remove_parentheses(lines)
    lines= remove_lines_without_arrow(lines, config_CFD) #is this really needed???? CFDDiscovery I think guarantees implication direction =>
    

    #filterRules remove FDs and keep CFDs. #optional: we can also delete some rules that are not interested for us by
    #specifiying condrhs and condlhs values like "conditionslhs": ["age-range=15-30"] or [["income=<50K"], ["workclass=private"]]
    #But for now we set these condrhs and condlhs as empty in config!
    final = filterRules(lines, config_CFD, condlhs, condrhs, print_summary= False)
    
    dict_cfds = parse_rules_to_dict(final, print_summary=True)
    
    return dict_cfds


dict_cfds = filter_cfds(raw_cfds, config_CFD)

CFD discovery algorithm completed successfully! Output saved to c:\Users\deart\Desktop\Tesi\My_Work\outputs\raw_cfd.txt
Total number of dependencies in the dictionary: 96
The first 50 dependencies:

1) {'lhs': {'education-degree': 'Elementary'}, 'rhs': {'income': '<=50k'}}
2) {'lhs': {'education-degree': 'MiddleSchool'}, 'rhs': {'income': '<=50k'}}
3) {'lhs': {'age_range': '15-30'}, 'rhs': {'income': '<=50k'}}
4) {'lhs': {'age_range': '15-30', 'education-degree': 'Assoc'}, 'rhs': {'income': '<=50k'}}
5) {'lhs': {'education-degree': 'MiddleSchool', 'age_range': '15-30'}, 'rhs': {'income': '<=50k'}}
6) {'lhs': {'education-degree': 'HS-College', 'age_range': '15-30'}, 'rhs': {'income': '<=50k'}}
7) {'lhs': {'hours-per-week_range': '0-20'}, 'rhs': {'income': '<=50k'}}
8) {'lhs': {'hours-per-week_range': '0-20', 'education-degree': 'Assoc'}, 'rhs': {'income': '<=50k'}}
9) {'lhs': {'hours-per-week_range': '21-40', 'education-degree': 'MiddleSchool'}, 'rhs': {'income': '<=50k'}}
10) {'lhs': {